# IMPORTING USEFUL LIBRARIES AND DATASETS

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px
from datetime import datetime, date, timedelta
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/brazilian-ecommerce/olist_sellers_dataset.csv
/kaggle/input/brazilian-ecommerce/olist_customers_dataset.csv
/kaggle/input/brazilian-ecommerce/olist_orders_dataset.csv
/kaggle/input/brazilian-ecommerce/olist_products_dataset.csv
/kaggle/input/brazilian-ecommerce/olist_order_payments_dataset.csv
/kaggle/input/brazilian-ecommerce/olist_geolocation_dataset.csv
/kaggle/input/brazilian-ecommerce/olist_order_reviews_dataset.csv
/kaggle/input/brazilian-ecommerce/olist_order_items_dataset.csv
/kaggle/input/brazilian-ecommerce/product_category_name_translation.csv
/kaggle/input/marketing-funnel-olist/olist_closed_deals_dataset.csv
/kaggle/input/marketing-funnel-olist/olist_marketing_qualified_leads_dataset.csv


<h1>INTRODUCTION</h1>
Hello, we are researchers at the Mauá Technology Institute, in Brazil. In this project we will present some objectives that were defined from the crossing of two datasets provided by Olist Store.<br>


by [Gabriel Lima](http://https://github.com/GabrielLima1995),  [Giovanni Schiffini](https://github.com/schiffini), [Marcio Oliverio](https://github.com/oliverra), [Michelle Alves](https://github.com/MichelleAlves84) e Rafael Brayner Costa.



<h1>WHAT IS OLIST?</h1>
Olist is a SaaS (Software as a Service) technology company founded in 2015 that offers a solution to increase sales for retailers of all sizes, for most segments, whether they have an online presence or not.

Olist operates in the e-commerce segment, but it is not e-commerce itself. Olist is a large department store within the marketplaces, formed by thousands of other stores throughout Brazil.

This store is already connected to the main e-commerces in Brazil and offers these privileged spaces so that the participants can advertise their products to the e-commerces visitors.

<h1>COLLECTING INFORMATION ABOUT OLIST</h1>
This study used the datasets:

*Marketing Funnel:* dataset with information from 8 thousand Marketing Qualified Leads (MQLs) who requested contact with Olist between June 1, 2017 and June 1, 2018. This dataset shows the sales process in multiple dimensions: lead category, catalog size, behavior profile, etc.

*Brazilian E-Commerce Public Dataset:* dataset with information on 100 thousand orders from 2016 to 2018 placed in different markets in Brazil. Its features allow the viewing of an order in multiple dimensions: from order status, price, payment and shipping performance to the customer's location, product attributes and, finally, written reviews by customers. There is also a set of geolocation data that relates Brazilian postal codes to lat / lng coordinates.

All data exposed in these two datasets are real and have been made anonymous and sampled from the original data set.

[](http://)<h1>OBJECTIVE</h1>

Find the best represetant for each deal! For this, probably you will have to create a model capable of using the information from the seller_id in the Brazillian OList ECommerce database to characterize each particular seller, and their products, to finally find the best cluster of representants to use in a particular client/seller.

Importing Dataset 

In [2]:
marketing_qualified_leads = pd.read_csv("/kaggle/input/marketing-funnel-olist/olist_marketing_qualified_leads_dataset.csv")
closed_deals_dataset      = pd.read_csv("/kaggle/input/marketing-funnel-olist/olist_closed_deals_dataset.csv")
order_itens_dataset       = pd.read_csv("/kaggle/input/brazilian-ecommerce/olist_order_items_dataset.csv")
df_products               = pd.read_csv("/kaggle/input/brazilian-ecommerce/olist_products_dataset.csv")

Checking the headers of each dataset

In [3]:
marketing_qualified_leads.head()

,mql_id,first_contact_date,landing_page_id,origin
0,dac32acd4db4c29c230538b72f8dd87d,2018-02-01,88740e65d5d6b056e0cda098e1ea6313,social
1,8c18d1de7f67e60dbd64e3c07d7e9d5d,2017-10-20,007f9098284a86ee80ddeb25d53e0af8,paid_search
2,b4bc852d233dfefc5131f593b538befa,2018-03-22,a7982125ff7aa3b2054c6e44f9d28522,organic_search
3,6be030b81c75970747525b843c1ef4f8,2018-01-22,d45d558f0daeecf3cccdffe3c59684aa,email
4,5420aad7fec3549a85876ba1c529bd84,2018-02-21,b48ec5f3b04e9068441002a19df93c6c,organic_search


In [4]:
closed_deals_dataset.head()

,mql_id,seller_id,sdr_id,sr_id,won_date,business_segment,lead_type,lead_behaviour_profile,has_company,has_gtin,average_stock,business_type,declared_product_catalog_size,declared_monthly_revenue
0,5420aad7fec3549a85876ba1c529bd84,2c43fb513632d29b3b58df74816f1b06,a8387c01a09e99ce014107505b92388c,4ef15afb4b2723d8f3d81e51ec7afefe,2018-02-26 19:58:54,pet,online_medium,cat,NaN,NaN,NaN,reseller,NaN,0.0
1,a555fb36b9368110ede0f043dfc3b9a0,bbb7d7893a450660432ea6652310ebb7,09285259593c61296eef10c734121d5b,d3d1e91a157ea7f90548eef82f1955e3,2018-05-08 20:17:59,car_accessories,industry,eagle,NaN,NaN,NaN,reseller,NaN,0.0
2,327174d3648a2d047e8940d7d15204ca,612170e34b97004b3ba37eae81836b4c,b90f87164b5f8c2cfa5c8572834dbe3f,6565aa9ce3178a5caf6171827af3a9ba,2018-06-05 17:27:23,home_appliances,online_big,cat,NaN,NaN,NaN,reseller,NaN,0.0
3,f5fee8f7da74f4887f5bcae2bafb6dd6,21e1781e36faf92725dde4730a88ca0f,56bf83c4bb35763a51c2baab501b4c67,d3d1e91a157ea7f90548eef82f1955e3,2018-01-17 13:51:03,food_drink,online_small,NaN,NaN,NaN,NaN,reseller,NaN,0.0
4,ffe640179b554e295c167a2f6be528e0,ed8cb7b190ceb6067227478e48cf8dde,4b339f9567d060bcea4f5136b9f5949e,d3d1e91a157ea7f90548eef82f1955e3,2018-07-03 20:17:45,home_appliances,industry,wolf,NaN,NaN,NaN,manufacturer,NaN,0.0


In [5]:
order_itens_dataset.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [7]:
df_funnel = marketing_qualified_leads.merge(closed_deals_dataset, on='mql_id', how='left')

In [8]:
df_funnel.head()

,mql_id,first_contact_date,landing_page_id,origin,seller_id,sdr_id,sr_id,won_date,business_segment,lead_type,lead_behaviour_profile,has_company,has_gtin,average_stock,business_type,declared_product_catalog_size,declared_monthly_revenue
0,dac32acd4db4c29c230538b72f8dd87d,2018-02-01,88740e65d5d6b056e0cda098e1ea6313,social,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8c18d1de7f67e60dbd64e3c07d7e9d5d,2017-10-20,007f9098284a86ee80ddeb25d53e0af8,paid_search,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,b4bc852d233dfefc5131f593b538befa,2018-03-22,a7982125ff7aa3b2054c6e44f9d28522,organic_search,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6be030b81c75970747525b843c1ef4f8,2018-01-22,d45d558f0daeecf3cccdffe3c59684aa,email,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5420aad7fec3549a85876ba1c529bd84,2018-02-21,b48ec5f3b04e9068441002a19df93c6c,organic_search,2c43fb513632d29b3b58df74816f1b06,a8387c01a09e99ce014107505b92388c,4ef15afb4b2723d8f3d81e51ec7afefe,2018-02-26 19:58:54,pet,online_medium,cat,NaN,NaN,NaN,reseller,NaN,0.0


# <b>UNDERSTANDING THE DATA</b>

Olist Dataset Schema

![](https://i.imgur.com/HRhd2Y0.png)

Marketing Funnel Dataset Schema

![](https://i.imgur.com/Jory0O3.png)

# DESCRIPTION OF THE COLUMNS

**marketing_qualified_leads**</p>
*Columns | mql_id	| first_contact_date | landing_page_id | origin* 
<p></p>
<p>mql_id : Marketing Qualified Lead id </p>
<p>first_contact_date : Date of first contact between seller and customer  </p>
<p>landing_page_ID : Landing page id where the lead was acquired  </p>
<p>origin : Type of media where the lead was acquired </p>

**closed_deals_dataset**</p>
*Columns | mql_id |	seller_id | sdr_id |	sr_id |	won_date  |	business_segment  |	lead_type |	lead_behaviour_profile |	has_company |	has_gtin  |	average_stock |	business_type |	declared_product_catalog_size	| declared_monthly_revenue*
<p></p>


<p>mql_id : Marketing Qualified Lead ID</p>
<p>seller_id : Seller ID  </p>
<p>sdr_id: Sales development Representative ID </p>
<p>sr_id: Sales Representative ID </p>
<p>won_date: Date the deal was closed </p>
<p>business_segment: Lead business segment (Informed on contact) </p>
<p>lead_type: Lead type (Informed on contact) </p>
<p>lead_behaviour_profile : Lead behaviour profile (SDR identify it on contact)</p>
<p>has_company: Does the lead have a company (formal documentation)? </p>
<p>has_gtin: Does the lead have Global Trade Item Number (barcode) for his products? </p>
<p>average_stock: Lead declared average stock (Informed on contact) </p>
<p>business_type: Type of business (reseller/manufacturer etc.) </p>
<p>declared_product_catalog_size: Lead declared catalog size (Informed on contact) </p>
<p>declared_monthly_revenue: Lead declared estimated monthly revenue (Informed on contact)  </p>

**order_itens_dataset**</p>
*Columns | order_id | order_item_id | product_id | seller_id | shipping_limit_date | price | freight_value*
<p></p>


<p>order_id: sequential number identifying number of items included in the same order </p>
<p>product_id: Product unique identifier</p>
<p>seller_id: Seller unique identifier</p>
<p>shipping_limit_date: Shows the seller shipping limit date for handling the order over to the logistic partner</p>
<p>price: Item price </p>
<p>freight_value: Item freight value item (if an order has more than one item the freight value is splitted between items) </p>

**df_funnel** </p>

Merge between *marketing_qualified_leads* and *closed_deals_dataset*
<p></p>


# ANALYSING THE DATA

Merging the *order_itens_dataset* from *seller_id*. The "left" method inserts the new data at the last columns. The *product_id* has been dropped because it was already at the dataframe.

In [9]:
df_funnel_items = df_funnel.dropna(subset=["seller_id"]).merge(order_itens_dataset, on='seller_id', how='left')
df_funnel_items = df_funnel_items.dropna(subset=["product_id"])

In [10]:
df_funnel_items.head()

,mql_id,first_contact_date,landing_page_id,origin,seller_id,sdr_id,sr_id,won_date,business_segment,lead_type,...,average_stock,business_type,declared_product_catalog_size,declared_monthly_revenue,order_id,order_item_id,product_id,shipping_limit_date,price,freight_value
0,5420aad7fec3549a85876ba1c529bd84,2018-02-21,b48ec5f3b04e9068441002a19df93c6c,organic_search,2c43fb513632d29b3b58df74816f1b06,a8387c01a09e99ce014107505b92388c,4ef15afb4b2723d8f3d81e51ec7afefe,2018-02-26 19:58:54,pet,online_medium,...,NaN,reseller,NaN,0.0,1efcc6ba75b519535a510c53153a9256,1.0,e94df75b2e9a8b056be0c144a731f55c,2018-06-26 22:20:21,94.2,18.76
1,5420aad7fec3549a85876ba1c529bd84,2018-02-21,b48ec5f3b04e9068441002a19df93c6c,organic_search,2c43fb513632d29b3b58df74816f1b06,a8387c01a09e99ce014107505b92388c,4ef15afb4b2723d8f3d81e51ec7afefe,2018-02-26 19:58:54,pet,online_medium,...,NaN,reseller,NaN,0.0,d8d281c1a1398a1e835fdc4adfb81e2a,1.0,9e0873e9ac04b12f282b81af5daebd2e,2018-07-30 09:31:49,664.0,38.41
2,5420aad7fec3549a85876ba1c529bd84,2018-02-21,b48ec5f3b04e9068441002a19df93c6c,organic_search,2c43fb513632d29b3b58df74816f1b06,a8387c01a09e99ce014107505b92388c,4ef15afb4b2723d8f3d81e51ec7afefe,2018-02-26 19:58:54,pet,online_medium,...,NaN,reseller,NaN,0.0,e5858127da61f07285827883e605254b,1.0,9bb2d066e4b33b624cbdfec7d50b3dcb,2018-06-27 16:00:04,99.8,15.00
4,327174d3648a2d047e8940d7d15204ca,2018-04-03,1ceb590cd1e00c7ee95220971f82693d,organic_search,612170e34b97004b3ba37eae81836b4c,b90f87164b5f8c2cfa5c8572834dbe3f,6565aa9ce3178a5caf6171827af3a9ba,2018-06-05 17:27:23,home_appliances,online_big,...,NaN,reseller,NaN,0.0,0475d054d40280dfba750663c9ad7a0a,1.0,a86859b91d632c991b914aead6ec8ad4,2018-08-07 17:35:18,49.9,13.69
5,327174d3648a2d047e8940d7d15204ca,2018-04-03,1ceb590cd1e00c7ee95220971f82693d,organic_search,612170e34b97004b3ba37eae81836b4c,b90f87164b5f8c2cfa5c8572834dbe3f,6565aa9ce3178a5caf6171827af3a9ba,2018-06-05 17:27:23,home_appliances,online_big,...,NaN,reseller,NaN,0.0,05104976125ea1e5cc04bc1f506a436a,1.0,63bfa27a0cee21fd896dc27f58b6d656,2018-08-19 22:44:03,198.9,18.32


Now, the *df_products* dataset is merged from the *product_id* column, again attaching its columns to the last positions.

In [11]:
df_funnel_products = df_funnel_items.merge(df_products, on="product_id", how="left")

In [12]:
df_funnel_products.head()

,mql_id,first_contact_date,landing_page_id,origin,seller_id,sdr_id,sr_id,won_date,business_segment,lead_type,...,price,freight_value,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,5420aad7fec3549a85876ba1c529bd84,2018-02-21,b48ec5f3b04e9068441002a19df93c6c,organic_search,2c43fb513632d29b3b58df74816f1b06,a8387c01a09e99ce014107505b92388c,4ef15afb4b2723d8f3d81e51ec7afefe,2018-02-26 19:58:54,pet,online_medium,...,94.2,18.76,pet_shop,56.0,1538.0,2.0,625.0,23.0,17.0,19.0
1,5420aad7fec3549a85876ba1c529bd84,2018-02-21,b48ec5f3b04e9068441002a19df93c6c,organic_search,2c43fb513632d29b3b58df74816f1b06,a8387c01a09e99ce014107505b92388c,4ef15afb4b2723d8f3d81e51ec7afefe,2018-02-26 19:58:54,pet,online_medium,...,664.0,38.41,ferramentas_jardim,25.0,432.0,1.0,2825.0,29.0,29.0,29.0
2,5420aad7fec3549a85876ba1c529bd84,2018-02-21,b48ec5f3b04e9068441002a19df93c6c,organic_search,2c43fb513632d29b3b58df74816f1b06,a8387c01a09e99ce014107505b92388c,4ef15afb4b2723d8f3d81e51ec7afefe,2018-02-26 19:58:54,pet,online_medium,...,99.8,15.00,pet_shop,53.0,1494.0,2.0,900.0,26.0,19.0,23.0
3,327174d3648a2d047e8940d7d15204ca,2018-04-03,1ceb590cd1e00c7ee95220971f82693d,organic_search,612170e34b97004b3ba37eae81836b4c,b90f87164b5f8c2cfa5c8572834dbe3f,6565aa9ce3178a5caf6171827af3a9ba,2018-06-05 17:27:23,home_appliances,online_big,...,49.9,13.69,sinalizacao_e_seguranca,60.0,1170.0,1.0,450.0,16.0,5.0,16.0
4,327174d3648a2d047e8940d7d15204ca,2018-04-03,1ceb590cd1e00c7ee95220971f82693d,organic_search,612170e34b97004b3ba37eae81836b4c,b90f87164b5f8c2cfa5c8572834dbe3f,6565aa9ce3178a5caf6171827af3a9ba,2018-06-05 17:27:23,home_appliances,online_big,...,198.9,18.32,eletronicos,59.0,600.0,6.0,475.0,20.0,11.0,19.0


Changing the *won_date* (deal closing date) to a timestamp, using the datetime library. This action makes it easy to order and filter by date.

In [13]:
df_funnel_products['won_date'][0]

'2018-02-26 19:58:54'

In [14]:
df_funnel_products["won_date"] = df_funnel_products["won_date"].apply(lambda d: datetime.strptime(d,"%Y-%m-%d %H:%M:%S"))

In [15]:
df_funnel_products['won_date'][0]

Timestamp('2018-02-26 19:58:54')

Now, the *first_contact_date* will be transformed into a timestamp.

In [16]:
df_funnel_products["first_contact_date"] = df_funnel_products["first_contact_date"].apply(lambda d: datetime.strptime(d,"%Y-%m-%d"))

In [17]:
df_funnel_products.seller_id.unique().shape[0]

380

As we can see above, the dataframe has 380 unique seller ids, so, 380 individual sellers are represented in this dataframe.

In [18]:
df_funnel_products.business_segment.unique()

array(['pet', 'home_appliances', 'health_beauty', 'household_utilities',
       'construction_tools_house_garden', 'sports_leisure', 'food_drink',
       'home_decor', 'watches', 'computers', 'small_appliances',
       'audio_video_electronics', 'bed_bath_table', 'car_accessories',
       'home_office_furniture', 'books', 'party', 'air_conditioning',
       'phone_mobile', 'music_instruments', 'baby', 'handcrafted',
       'fashion_accessories', 'toys', 'bags_backpacks', 'stationery',
       'food_supplement', 'gifts', 'games_consoles'], dtype=object)

There are 28 business segments in this dataframe.

Using the *plotly* library, the following chart shows the main business segments on the OList Brazilian ecommerce.

In [19]:
analisys  = df_funnel_products.business_segment.value_counts()

In [21]:
fig = px.pie(values=analisys, names=analisys.index, title='Most common business segments ')
fig.show()

Since *health and beauty* is the most relevant business segment, we are going to look specifically at its data:

# **A close look at the 'Health and beauty' business segment**

In [22]:
health_beauty = df_funnel_products.where(df_funnel_products['business_segment'] == 'health_beauty').dropna(subset=["seller_id"])


In [23]:
health_beauty.seller_id.unique().shape[0]

45

Selecting specifically the *health_beauty* business segment with the 'dropna' pandas funcion, there are only 45 sellers.

# Sales representative with the most sales:

In [24]:
fig = px.histogram(health_beauty.seller_id, x="seller_id").update_xaxes(categoryorder="total descending")
fig.show()

Three sales representatives are responsible for most of the deals in this segment, specially the first one. This could be one method to determine which is the best seller representative, but other methods are showed on the next steps.

Down below, the list of all sellers in this segment and their *first_contact_date*.

In [52]:
health_beauty_init_date    = health_beauty.groupby(by = 'seller_id').min()['first_contact_date']
health_beauty_oldest_seller= health_beauty_init_date.sort_values(axis=0, ascending=True)
health_beauty_oldest_seller

seller_id
89a51f50b8095ea78d5768f34c13a76f   2017-08-30
4a82168aaef173cbb6e46ae88728f919   2017-12-26
c54679b132454625907c773d273d4126   2018-01-03
bd4431c1a8dfa744b59e1e2a8bc99b9e   2018-01-03
4867209cb29b56b92893d7bde3ce8012   2018-01-11
e116c7455dd26a4d8c3e92532583905f   2018-01-12
2fa13c8bd5705d279f7ed5cc9ec61c68   2018-01-12
0b1ca3ef18a63d7eb0c8897fa0849c08   2018-01-17
dace965ca58120f92f8d742a9fa1864b   2018-01-20
ae45df84722b1d15c5f32b23a095746c   2018-01-24
dbdd0ec73a4817971d962698f2fea022   2018-01-24
6aa3928696eab49a19a466e665a649fd   2018-02-02
1c742ac33582852aaf3bcfbf5893abcf   2018-02-06
4a23adac87cf538689fa600d976f78b2   2018-02-07
6bd503a8230299c9316fb0f726933927   2018-02-10
ca7c6bd577e559472af1c699de9e764e   2018-02-16
2e7ea2aafec5835e781b21d6d1faf1f1   2018-02-22
28c7d8743fbc8679f484868cc0fcaefb   2018-02-22
f22a247b79dd22ecd91dc8e3b2a1b07c   2018-02-23
4f0d637c1197fd65405ebd755ac9f86b   2018-02-28
094ced053e257ae8cae57205592d6712   2018-03-02
a69aa24cec11168e7e39a3fb

# Mean sales per day for each sales representative:

Grouping the sellers by its *won_date*.

In [26]:
health_beauty_won_date_max= health_beauty.groupby(by = 'seller_id').max()['won_date']

Subtracting the *initial date* from the *won date*, to determine how long it took, in average, to close a deal.

In [27]:
health_beauty_days_sellers = (health_beauty_won_date_max-health_beauty_init_date).apply(lambda d: d.days).to_frame(name='days_')

Counting the transactions on this business segment.

In [28]:
health_beauty_count = health_beauty.groupby(by = 'seller_id').count()['mql_id'].to_frame(name ='count')

Merging the sellers to the total transaction count dataframe.

In [29]:
health_beauty_days_sellers = health_beauty_days_sellers.merge(health_beauty_count, on='seller_id', how='left')


Obtaining the mean sales per day for each seller id.

In [30]:
health_beauty_days_sellers['mean'] = health_beauty_days_sellers['count'] /health_beauty_days_sellers['days_']

In [31]:
health_beauty_days_sellers.sort_values(axis=0, ascending=False,by='mean')

,days_,count,mean
seller_id,,,
c70c1b0d8ca86052f45a432a38b73958,0,338,inf
985c92cc412091f8529c12d7f23d3fef,0,12,inf
516e7738bd8f735ac19a010ee5450d8d,6,84,14.000000
698f5fa55a5f73d0740e9d9a773e8093,1,6,6.000000
6aa3928696eab49a19a466e665a649fd,5,22,4.400000
eb72802c83dc7547529c9546d1a9b8ef,2,8,4.000000
5670f4db5b62c43d542e1b2d56b0cf7c,6,22,3.666667
a63bfbaa882c8f4542891b4e2246cc7f,2,7,3.500000
f46490624488d3ff7ce78613913a7711,42,91,2.166667


In [32]:
fig = px.bar(health_beauty_days_sellers['mean'], x=health_beauty_days_sellers.index, y=['mean']).update_xaxes(categoryorder="total descending")
fig.show()

As we can see above, the mean sales per day is a better method to analyse this data. Some sales representatives may seem to sell less in the general terms, but they have a better sale/day ratio. The main sale representative, harshed as "c70c", is considered the first in both methods. Most of his transactions are won on the same day they begun, resulting in this "zero days" mean.

In [33]:
fig = px.scatter(health_beauty_days_sellers, x = "days_" , y = "count")
fig.show()

Most of the sales are won on the very first day or within a week. However, some sales take more than 200 days to be confirmed, which is highly illogical.

# **Watches business segment.**

Down below, the same analysis is made on the Watches segment, which is the second most important on this dataframe.

In [34]:
watches = df_funnel_products.where(df_funnel_products['business_segment'] == 'watches').dropna(subset=["seller_id"])

In [35]:
watches.seller_id.unique().shape[0]

3

There are only 3 sellers in this segment.

In [36]:
fig = px.histogram(watches.seller_id, x="seller_id").update_xaxes(categoryorder="total descending")
fig.show()

In [37]:
watches_init_date    = watches.groupby(by = 'seller_id').min()['first_contact_date']
watches_oldest_seller= watches_init_date.sort_values(axis=0, ascending=True)
watches_oldest_seller

seller_id
7d13fca15225358621be4086e1eb0964   2018-01-22
f9eda05b67bef472deaddbba84aca289   2018-02-26
bce6a5ecf98e99fd2a8868b15c51f04b   2018-04-12
Name: first_contact_date, dtype: datetime64[ns]

In [38]:
watches_won_date_max= watches.groupby(by = 'seller_id').max()['won_date']

In [39]:
watches_days_sellers = (watches_won_date_max-watches_init_date).apply(lambda d: d.days).to_frame(name='days_')

In [40]:
watches_count = watches.groupby(by = 'seller_id').count()['mql_id'].to_frame(name ='count')

In [41]:
watches_days_sellers = watches_days_sellers.merge(watches_count, on='seller_id', how='left')

In [42]:
watches_days_sellers['mean'] = watches_days_sellers['count'] /watches_days_sellers['days_']

In [43]:
watches_days_sellers.sort_values(axis=0, ascending=False,by='mean')

,days_,count,mean
seller_id,,,
7d13fca15225358621be4086e1eb0964,17,578,34.000000
bce6a5ecf98e99fd2a8868b15c51f04b,1,15,15.000000
f9eda05b67bef472deaddbba84aca289,9,1,0.111111


In [44]:
fig = px.bar(watches_days_sellers['mean'], x=watches_days_sellers.index, y=['mean']).update_xaxes(categoryorder="total descending")
fig.show()

In this case, the sales representatives which sold the most are also the ones with the best sales/day ratio.